In [2]:
import pandas as pd
from time import sleep
import re
import threading
import random
from selenium import webdriver # 从selenium导入webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import os
from itertools import product

colleges = pd.read_excel('高校名单.xlsx',sheet_name='统计学')
colleges = set(colleges['高校名称'])
index_url = 'http://www.letpub.com.cn/index.php?page=grant'
with open('项目数据/国家自然-统计学-经济学/succeed.txt','r',encoding='utf-8') as f:
    data= f.readlines()
succeed = [x for x in data]

In [4]:
for col in colleges:
    alldata = []
    if col in succeed:
        print(col + '之前采集完毕')
        continue
    driver = webdriver.Chrome()
    driver.get(index_url)
    sleep(1)
    ad = driver.find_element_by_class_name('layui-layer-setwin').find_element_by_tag_name('a')
    ad.click() #关掉广告
    unit = driver.find_element_by_id('company')
    unit.send_keys(col)
    sleep(1)
    Select(driver.find_element_by_id('addcomment_s1')).select_by_value('G') # 学科类别 G管理学部 A数理科学
    #Select(driver.find_element_by_id('addcomment_s2')).select_by_value('G02') # 02工商管理，03经济科学
    Select(driver.find_element_by_id('addcomment_s2')).select_by_value('G03') # 一级学科 02工商管理，03经济科学
    Select(driver.find_element_by_id('addcomment_s3')).select_by_value('G0303') #二级学科    
    Select(driver.find_element_by_id('startTime')).select_by_value('2016') #  开始年限，（截止年限默认2018）
    sleep(1)
    driver.find_element_by_id('submit').click()
    sleep(3)
    center = driver.find_element_by_tag_name('center').text
    counts = re.findall(r'共(\d+)页',center)[0]
    if int(counts) == 0:
        print(col + '没有检索结果')
        succeed.append(col)
        driver.quit()
        continue
    for i in range(int(counts)):
        all_message = driver.find_element_by_class_name('table_yjfx').find_elements_by_tag_name('td')[1:] #去掉开头结尾的td
        i = 0
        one_item = []
        for item in all_message:
            if i < 15:
                one_item.append(item.text)
                i+=1
            else:
                i = 0
                alldata.append(one_item) #汇总
                one_item = []
                one_item.append(item.text)#15
                i+=1
        next_page = driver.find_element_by_link_text(u'下一页') # 下一页
        next_page.click()
    #结束该学校
    driver.quit()
    columns = ['负责人','单位','金额 (万)','项目编号','项目类型','所属学部','批准年份','题目','题目','学科分类','学科分类','学科代码','学科代码','执行时间','执行时间']
    df = pd.DataFrame(alldata,columns=columns)
    df.to_excel('项目数据/国家自然-统计学-经济学/' + col + '.xlsx' )
    print(col+' ok')
    succeed.append(col)

with open('项目数据/国家自然-统计学-经济学/succeed.txt','w',encoding='utf-8') as f:
    for item in succeed:
        f.write(item)
        f.write('\n')
print('全部采集完毕')

吉林大学 ok
中南财经政法大学没有检索结果
山东大学没有检索结果
江西财经大学没有检索结果
首都师范大学没有检索结果
清华大学 ok
首都经济贸易大学 ok
西南财经大学没有检索结果
北京工业大学没有检索结果
浙江工商大学没有检索结果
中南大学没有检索结果
中央财经大学没有检索结果
北京交通大学没有检索结果
对外经贸大学没有检索结果
东北财经大学 ok
复旦大学 ok
全部采集完毕


In [2]:
alldata

NameError: name 'alldata' is not defined

In [21]:
columns = ['负责人','单位','金额 (万)','项目编号','项目类型','所属学部','批准年份','题目','题目','学科分类','学科分类','学科代码','学科代码','执行时间','执行时间']
df = pd.DataFrame(alldata,columns=columns)
df.to_excel('国家自然科学基金项目.xlsx')

In [28]:
df['单位'].value_counts()

山东财经大学      27
山西财经大学      18
江西财经大学       9
华中科技大学       9
山东大学         9
吉林大学         9
南京大学         9
首都经济贸易大学     9
中南大学         9
天津财经大学       9
西南财经大学       9
武汉大学         9
北京工业大学       9
清华大学         9
中央财经大学       9
浙江大学         9
湖南大学         9
浙江财经大学       9
中南财经政法大学     9
浙江工商大学       9
复旦大学         9
北京交通大学       9
中山大学         9
首都师范大学       9
东北财经大学       9
辽宁大学         9
Name: 单位, dtype: int64

In [26]:
len(colleges)

27